# Library 정보 수집 

In [34]:
import requests
from enum import Enum
import pandas as pd
from pandas.io.json import json_normalize
import xmltodict
import json
import cx_Oracle
from datetime import datetime
import schedule
import time
from stylecloud import gen_stylecloud
import os

myKey = 'af2df55209c61c0726e45405de83247e14d2474c9c0bca948854d5495b6465a1'
isbnList = []

class LibCode(Enum):
    Code01 = 124004
    Code02 = 124012
    Code03 = 129224
    Code04 = 129219
    Code05 = 124007
    

def initDB():
    # 최신데이터로 업데이트하기 위하여 기존 데이터 삭제
    connect = cx_Oracle.connect("Library_admin", "12345", "localhost:1521/xe")
    cursor = connect.cursor()
    sql_delete = 'delete from book'
    cursor.execute(sql_delete)
    cursor.close()
    connect.commit()
    connect.close()
    print('[' + datetime.now().strftime('%Y%m%d') + ']','DB 초기화 완료.....')
    
def collectionBook():
    global isbnList
    oriUrl = 'http://data4library.kr/api/loanItemSrchByLib'
    region = '24'  #광주
    dtl_region = '24030'  #광주 남구
    pageSize = '10'
    count = 0
    checkDB = False
    if isbnList:
        isbnList = []
    for code in LibCode:
        url = oriUrl + "?authKey=" + myKey + '&startDt=2022-01-01&endDt=2022-11-21' + \
            '&pageNo=1&pageSize=' + pageSize + "&libCode=" + str(code.value)
        response = requests.get(url)
        
        # XML 수집
        content = response.text
        
        # XML -> JSON 변환
        result = xmltodict.parse(content)
        dictionay = json.loads(json.dumps(result))
        if dictionay != "" and checkDB == False:
            initDB()
            checkDB = True
        elif checkDB == True:
            pass
        else:
            print('[' + datetime.now().strftime('%Y%m%d') + ']','데이터 수집 오류 발생.....')
            return
        connect = cx_Oracle.connect("Library_admin", "12345", "localhost:1521/xe")
        cursor = connect.cursor()
        df = pd.json_normalize(dictionay['response']['docs']['doc'])
        columns = ['bookname','authors','isbn13','bookImageURL']
        df = df[columns]
        isbnList += df['isbn13'].values.tolist();
        for i in df.index:
            sql_insert = 'INSERT INTO BOOK(book_number,bookname,authors,isbn13,book_ImageURL)' +\
            'VALUES(:booknumber, :bookname, :authors, :isbn13, :bookImageURL)'
            
            booknumber = count
            bookname = df.loc[i].bookname
            authors = df.loc[i].authors
            isbn13 = df.loc[i].isbn13
            bookImageURL = df.loc[i].bookImageURL
            
            cursor.execute(sql_insert, (booknumber, bookname, authors, isbn13, bookImageURL))
            connect.commit()
            count += 1
        connect.close()
    
    print('[' + datetime.now().strftime('%Y%m%d') + ']','DB 데이터 입력 완료.....')
    
    
def bookKeyWord():
    global isbnList
    main_dir_path = "C:/BookLibrary"
    oriUrl = 'http://data4library.kr/api/keywordList'
    path = 'C:\Windows\Fonts\HMFMPYUN.TTF'
    
    if not os.path.isdir(main_dir_path):
        print('폴더를 생성합니다.')
        os.makedirs(main_dir_path)
            
    
    if os.path.exists(main_dir_path):
        print('기존 이미지를 삭제합니다.')
        for file in os.scandir(main_dir_path):
            os.remove(file.path)
    else:
        print('폴더가 존재하지 않습니다.')
            
    for isbn in isbnList:
        url = oriUrl + "?authKey=" + myKey + "&isbn13=" + isbn + "&format=json"
        response = requests.get(url)
        content = response.json()
        
        if content['response']['resultNum'] < 50:
            print('isbn : ' + str(isbn) +' / 데이터가 존재하지 않습니다.')
            fileName = 'C:/BookLibrary/' + isbn + '.png'
            gen_stylecloud(text="X",
                          font_path=path,
                          icon_name="fas fa-square",
                          palette="colorbrewer.sequential.YlOrBr_3",
                          background_color='black',
                          gradient="horizontal",
                          output_name=fileName)
            continue
        df = pd.json_normalize(content['response']['items'])
        df["item.weight"] = df["item.weight"].astype(int)
        wc_data = df.set_index("item.word").to_dict()["item.weight"]
        
        
        fileName = 'C:/BookLibrary/' + isbn + '.png'
        gen_stylecloud(text=wc_data,
                          font_path=path,
                          icon_name="fas fa-square",
                          palette="colorbrewer.sequential.YlOrBr_3",
                          background_color='black',
                          gradient="horizontal",
                          output_name=fileName)
    print('[' + datetime.now().strftime('%Y%m%d') + ']','이미지 처리 완료.....')
    
    
collectionBook()
bookKeyWord()
# schedule.every().day.at("11:28:30").do(collectionBook)
# schedule.every().day.at("11:28:30").do(bookKeyWord)
# schedule.every().monday.at("13:30").do(collectionBook)
# schedule.every().monday.at("13:30").do(bookKeyWord)
# while True:
#     schedule.run_pending()
#     time.sleep(1)

[20221202] DB 초기화 완료.....
[20221202] DB 데이터 입력 완료.....
기존 이미지를 삭제합니다.
isbn : 9791164135158 / 50
isbn : 9791164134649 / 50
isbn : 9791161571188 / 50
isbn : 9791164134847 / 50
isbn : 9788949162041 / 50
isbn : 9791165341909 / 50
isbn : 9791168410848 / 50
isbn : 9791164063734 / 50
isbn : 9788936448011 / 50
isbn : 9791164136261 / 50
isbn : 9791190630016 / 50
isbn : 9791190305891 / 50
isbn : 9791164137053 / 50
isbn : 9791189938352 / 50
isbn : 9788930210959 / 50
isbn : 9791130635712 / 50
isbn : 9791190182522 / 50
isbn : 9791190786805 / 50
isbn : 9788950973353 / 50
isbn : 9791164062072 / 50
isbn : 9791195794799 / 50
isbn : 9791164063581 / 50
isbn : 9788937884931 / 50
isbn : 9788937899232 / 50
isbn : 9788950982898 / 50
isbn : 9788937883309 / 50
isbn : 9788950984748 / 50
isbn : 9791164062898 / 50
isbn : 9788937883316 / 50
isbn : 9788937899010 / 50
isbn : 9791164062898 / 50
isbn : 9791168410848 / 50
isbn : 9791190980333 / 50
isbn : 9788950981150 / 50
isbn : 9791164131686 / 50
isbn : 9791164136032